In [1]:
import json
import numpy
import pandas
from datetime import datetime
from pprint import pprint
pandas.options.display.max_colwidth = -1
print('done')

done


In [2]:
# runs in about 5 minutes
with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')
print('done')

ok
done


In [3]:
#allgames_df.sample(10).index
#mygames = ['NYN201204100']
mygames = ['BAL201208060']

#mygames += ['SEA201208150', 'BAL201208300', 'BAL201209300', 'BAL201304210']
#mygames += ['BAL201305170', 'BAL201405010', 'BAL201406130', 'BAL201406250', 'MIA201407030', 'PIT201605180']
#mygames += ['SDN201606150', 'WAS201607200', 'BAL201608180']

#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

somecolumns = allgames_df.columns
allgames2_df = pandas.DataFrame(columns=somecolumns)
allgames2_df['id'] = ''
allgames2_df = allgames2_df.set_index('id')
allgames2_df
for thisid, thisrow in allgames_df.iterrows():
    if (thisid in mygames):
        allgames2_df = allgames2_df.append(thisrow)
allgames_df = allgames2_df
print('done')

done


In [21]:
def load_rosters():
    all_rosters_df = pandas.read_csv('fullroster.csv')
    return all_rosters_df

def get_starters(onerow):
    
#    df = pandas.DataFrame()
    
    vsdf = pandas.DataFrame(onerow['visiting_starters'], columns=player_columns)
    vsdf['which_team'] = 0
    hsdf = pandas.DataFrame(onerow['home_starters'], columns=player_columns)
    hsdf['which_team'] = 1
    df = pandas.concat((vsdf, hsdf))
    df['batting_order'] = pandas.to_numeric(df['batting_order'])
    df['position'] = pandas.to_numeric(df['position'])

    return df

def split_event(df):
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['modifiers'] = df['modifiers'].fillna('')
    df['baserunning'] = df['modifiers'].str.split('.',n=1).str.get(1)
    df['baserunning'] = df['baserunning'].fillna('')
    df['modifiers'] = df['modifiers'].str.split('.').str.get(0)
    
    return df

def get_game_info(onerow, theindex):

    info_df = pandas.DataFrame.from_records(onerow['info'], index=theindex)

    teams_df = pandas.read_csv('teams.csv')
       
    # League is "A" for AL and "N" for NL
    info_df['visleague'] = teams_df['league'][teams_df['teamID'] == info_df['visteam'].tolist()[0]].tolist()[0][0:1]
    info_df['homeleague'] = teams_df['league'][teams_df['teamID'] == info_df['hometeam'].tolist()[0]].tolist()[0][0:1]

    info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
    info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
    info_df['dayofweek'] = info_df['date'].apply(lambda x: datetime.strftime(x, '%a'))
    
    info_df['gameno'] = pandas.to_numeric(info_df['number'], errors='coerce')
    info_df = info_df.drop('number', axis=1)
    info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
    info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
    info_df['timeofgame'] = pandas.to_numeric(info_df['timeofgame'], errors='coerce')
#    info_df['timeofgame'] = pandas.to_timedelta(info_df['timeofgame'].astype(str) + ' minutes', unit='m', errors='coerce')
    info_df = info_df.fillna('')

    return info_df

    
'''
    thisevent['game_date'] = this_info_df['date'].tolist()[0]
    thisevent['daynight'] = this_info_df['daynight'].tolist()[0]
    thisevent['parkID'] = this_info_df['site'].tolist()[0]
    thisevent['attendance'] = this_info_df['attendance'].tolist()[0]
    thisevent['timeofgame'] = this_info_df['timeofgame'].tolist()[0]

    thisevent['game_number'] = this_info_df['gameno'].tolist()[0]
    thisevent['visiting_team'] = this_info_df['visteam'].tolist()[0]
    thisevent['visiting_league'] = this_info_df['visleague'].tolist()[0]
    thisevent['home_team'] = this_info_df['hometeam'].tolist()[0]
    thisevent['home_league'] = this_info_df['homeleague'].tolist()[0]


    thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 1)].tolist()[0]
    thisevent['catcherID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 2)].tolist()[0]
    thisevent['1bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 3)].tolist()[0]
    thisevent['2bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 4)].tolist()[0]
    thisevent['3bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 5)].tolist()[0]
    thisevent['ssID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 6)].tolist()[0]
    thisevent['lfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 7)].tolist()[0]
    thisevent['cfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 8)].tolist()[0]
    thisevent['rfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 9)].tolist()[0]

    #                thisevent['dayofweek'] = 'xxx'
    thisevent['dayofweek'] = this_info_df['dayofweek'].tolist()[0]

    thisevent['visiting_gameno'] = -9999
    thisevent['home_gameno'] = -9999

    thisevent['visiting_score'] = -9999
    thisevent['home_score'] = -9999
    thisevent['length_outs'] = -9999

    thisevent['completed'] = -9999
    thisevent['forfeit'] = -9999
    thisevent['protest'] = -9999

    return thisevent
'''
print('done')

done


In [71]:
roster_df = load_rosters()

column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_batting_order', 'sub_position']

player_columns = ['playerID', 'name', 'batting_order', 'position']

display_order = ['gameID', 'visiting_team', 'inning', 'batting_team']
display_order += ['outs', 'balls', 'strikes', 'pitch_sequence']
display_order += ['visitor_score', 'home_score', 'batterID', 'batter_hand', 'pitcherID', 'pitcher_hand']

events_df = pandas.DataFrame()

for thisgameID,thisrow in allgames_df.iterrows():
    print(thisgameID)
    
    
    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
    
    this_info_df = get_game_info(thisrow, allgames_df.index)
        
    thisgame_df['gameID'] = thisgameID
        
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])

    playing_df = get_starters(thisrow)

    thisgame_df = split_event(thisgame_df)
    
    thisgame_df = thisgame_df.set_index('event_in_game')
    
    for ing in thisgame_df['inning'].drop_duplicates():
        for side in range(0,2):
            halfinn = thisgame_df[(thisgame_df['inning'] == ing) & (thisgame_df['home_or_visitor'] == side)]
            for eig, thisevent in halfinn.iterrows():
                thisevent['visiting_team'] = this_info_df['visteam'].tolist()[0]
                thisevent['home_team'] = this_info_df['hometeam'].tolist()[0]
                thisevent['inning'] = ing
                if (side == 0):
                    thisevent['batting_team'] = thisevent['visiting_team']
                    thisevent['fielding_team'] = thisevent['home_team']
                else:
                    thisevent['batting_team'] = thisevent['home_team']
                    thisevent['fielding_team'] = thisevent['visiting_team']
                #print(eig, thisevent['play'])
                thisevent['outs'] = -9999
                thisevent['balls'] = pandas.to_numeric(thisevent['pitch_count'][0:1], errors='coerce')
                thisevent['strikes'] = pandas.to_numeric(thisevent['pitch_count'][1:2], errors='coerce')
                thisevent['visitor_score'] = -9999
                thisevent['home_score'] = -9999

                # DON'T FORGET TO CORRECT THIS FOR BADJ IF NEEDED!
                thisevent['batter_hand'] = roster_df['batting_hand'][(roster_df['playerID'] == thisevent['batterID']) & (roster_df['team'] == thisevent['batting_team']) & (roster_df['year'] == float(thisevent['gameID'][3:7]))].tolist()[0]
#                print(thisevent['fielding_team'])
                fielding_flag = float(thisevent['fielding_team'] == thisevent['home_team'])
                thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 1)].tolist()[0]                               
 
                thisevent['pitcher_hand'] = roster_df['throwing_hand'][
                        (roster_df['playerID'] == thisevent['pitcherID']) & 
                        (roster_df['year'] == float(thisevent['gameID'][3:7])) 
                        ].tolist()[0]
                    
#                print(thisevent['pitcherID'])
                #&  (roster_df['team'] == thisevent['fielding_team'])]).tolist()[0]
#                thisevent['pitcher_hand'] = ''
#                thisevent['pitcher_hand'] = roster_df['throwing_hand'][(roster_df['playerID'] == thisevent['pitcherID']) & (roster_df['team'] == thisevent['fielding_team']) & (roster_df['year'] == float(thisevent['gameID'][3:7]))].tolist()[0]
                
                #    thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 1)].tolist()[0]
                #    thisevent['catcherID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 2)].tolist()[0]
                #    thisevent['1bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 3)].tolist()[0]
                #    thisevent['2bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 4)].tolist()[0]
#                thisevent['3bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 5)].tolist()[0]
#                thisevent['ssID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 6)].tolist()[0]
#                thisevent['lfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 7)].tolist()[0]
#                thisevent['cfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 8)].tolist()[0]
#                thisevent['rfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 9)].tolist()[0]


                
                events_df = events_df.append(thisevent)
#                print(thisevent[['batterID', 'play', 'vis']])
#                print(thisevent[['event_type', 'batterID', 'pitcherID', 'catcherID', '1bID', '2bID', '3bID', 'ssID', 'lfID', 'cfID', 'rfID', 'play', 'baserunning']])
#                print('----------')

events_df = events_df[display_order]
#roster_df.dtypes
#events_df[['batting_team', 'gameID']]
#events_df.dtypes

events_df.head(20)


BAL201208060


,gameID,visiting_team,inning,batting_team,outs,balls,strikes,pitch_sequence,visitor_score,home_score,batterID,batter_hand,pitcherID,pitcher_hand
0,BAL201208060,SEA,1.0,SEA,-9999.0,2.0,2.0,FBFBFS,-9999.0,-9999.0,ackld001,L,tillc001,R
1,BAL201208060,SEA,1.0,SEA,-9999.0,1.0,0.0,BX,-9999.0,-9999.0,saunm001,L,tillc001,R
2,BAL201208060,SEA,1.0,SEA,-9999.0,0.0,2.0,CCFX,-9999.0,-9999.0,montj003,R,tillc001,R
3,BAL201208060,SEA,1.0,BAL,-9999.0,0.0,1.0,CX,-9999.0,-9999.0,markn001,L,vargj001,L
4,BAL201208060,SEA,1.0,BAL,-9999.0,3.0,2.0,LBMBBS,-9999.0,-9999.0,andir001,R,vargj001,L
5,BAL201208060,SEA,1.0,BAL,-9999.0,1.0,0.0,BX,-9999.0,-9999.0,hardj003,R,vargj001,L
6,BAL201208060,SEA,1.0,BAL,-9999.0,1.0,2.0,S1SF>FBX,-9999.0,-9999.0,jonea003,R,vargj001,L
7,BAL201208060,SEA,1.0,BAL,-9999.0,1.0,0.0,BX,-9999.0,-9999.0,wietm001,B,vargj001,L
8,BAL201208060,SEA,2.0,SEA,-9999.0,0.0,0.0,X,-9999.0,-9999.0,jasoj001,L,tillc001,R
9,BAL201208060,SEA,2.0,SEA,-9999.0,2.0,2.0,BBCFC,-9999.0,-9999.0,seagk001,L,tillc001,R
